In [1]:
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os, os.path
import seaborn as sns



# <center> TP8 -  Reconnaissance de configurations manuelles sur des données de MoCap  </center>
<center> 2023/2024 - T. Godin, L. Naert </center>

<center> IUT de Vannes, BUT Informatique </center>



L’objectif de ce TP est de travailler sur des données de mouvements des mains (données de Motion Capture) et de faire de la reconnaissance de configurations manuelles sur chaque frame de ces données. Il s'agit d'une tache de classification pour de l'apprentissage automatique. Pour cela, vous utiliserez la librairie Python Sklearn.

Les données exploitées ici ont fait l’objet d’un protocole expérimental précis, d’enregistrements en salle MoCap, de post-traitements et d’une segmentation et annotation manuelle dans le laboratoire IRISA à Vannes (Tohannic). 

> __Question 1 (Mise en place)__ : Télécharger les données à étudier présentes sur Moodle et dézipper l'archive dans un dossier de même nom (PositionHandJoints) situé dans le même dossier que votre jupyter Notebook. 


## Capture de Mouvement 

La "Capture de mouvement" (ou "Motion Capture", ou "Mocap") est un procédé permettant d'acquérir des données de mouvements. Il existe plusieurs techniques pour faire de la MoCap mais ici, les données ont été acquises en plaçant des marqueurs réfléchissants sur la peau d'une personne. Le mouvement de ces marqueurs est capté par plusieurs caméras infrarouges et la position de chaque marqueur dans l'espace est calculée automatiquement par triangulation.

Il y a un gros travail de traitement à faire ensuite pour, partant de la position des marqueurs dans l'espace, en déduire la position des articulations. 

La figure ci-dessous donne un aperçu de la chaîne de traitement des données de MoCap : de l’identification des marqueurs au calcul des positions/orientations des articulations.

<img src="reconstructionSquelette.png" width="1000">


## Configuration de la main

Vous travaillerez sur des données de mouvements des mains. Ces données sont sous la forme de vecteurs de position des articulations des mains dans l’espace suivant 3 coordonnées (x, y et z). Nous avons défini 26 articulations par main (4 articulations par doigts + l’articulation du poignet). Chaque vecteur contient ainsi $3 * 26 = 78$ valeurs réelles correspondant chacune à une position suivant un axe pour une articulation.

La figure ci-dessous montre les 26 articulations définies pour ce TP.

<img src="main.png" width="500">

Une __configuration de la main__ correspond à une certaine disposition des doigts à un moment donné. La configuration ’5’, par exemple, correspond à un état de la main dans lequel tous les doigts sont tendus et écartés (faisant le chiffre 5) tandis que la configuration ’V’ sera faite en tendant et écartant l’index et le majeur (les autres doigts étant pliés). Nous avons répertorié 32 classes de configurations différentes. Celles-ci sont listées et illustrées sur la figure ci-dessous.

<img src="configManuelles.png" width="1000">

### Données


Nous avons extrait 600 configurations manuelles appartenant à ces 32 classes en nous basant
sur l’annotation d'un corpus de données existant. Ces configurations manuelles ont été exécutées durant la réalisation de phrases en Langue des Signes Française. Un fichier de type .csv correspondant à chacune de ces configurations a été créé. Chaque fichier suit le même schéma :
- la première ligne est un en-tête contenant la liste ordonnée des noms des articulations et d’un axe (x, y, ou z)
- les lignes suivantes décrivent la position des articulations pour chaque frame.

Chaque ligne (frame) est composée des 78 valeurs de position. Un fichier .csv correspond donc
à un ensemble de n frames consécutives auxquelles on a assigné la même étiquette.

Exemple du contenu d’un fichier pour n = 3 avec 2 articulations (poignet et première articulation
de l’auriculaire) :

`Left_Hand_x Left_Hand_y Left_Hand_z Left_Pinky0_x Left_Pinky0_y Left_Pinky0_z`

 `27 1.30285e-06 1.33303e-06 28.1445 2.15521 -2.85254`
 
 `27 -1.24972e-06 8.43971e-07 28.1407 2.13847 -2.86662`
 
 `27 -2.384e-06 -6.85626e-06 28.147 2.11799 -2.87928`
 
Pour récupérer les données, il faut lire les fichiers contenus dans le dossier PositionHandJoints. Les fichiers sont nommés de la manière suivante : `AH_Sx_fy_lz_label.csv`, avec :
- AH = LH ou RH en fonction de la main dont il est question : gauche (LH = Left Hand) ou droite (RH = Right Hand).
- x = numéro de la séquence de capture dont est extraite la configuration.
- y = frame de début.
- z = longueur de la séquence.
- label = étiquette du segment (valeur de la configuration manuelle).

__Exemple__ : 
`LH_S1_f6827_l61_Oui.csv` : Données de la main gauche (LH) extraites de la séquence demouvement "S1" commençant à la frame numéro 6827 d’une longueur de 78 frames et étiquetées "Oui" (appartenant à la classe ’Oui’ selon les annotateurs).



Le code suivant définit une classe pour stocker les métadonnées et données de chaque fichier de mouvement et permet de télécharger les données. 

__Attention : il y a une modification à faire si vous êtes sous Unix ! (voir ci-dessous)__

In [2]:
"""
Définition d'une classe PositionSequence qui stocke les métadonnees (infos du nom de fichier) 
et les données de mouvement elles-meme

Exemple d'un nom de fichier
LH_S1_f6827_l61_S.csv
key : LH_S1_f6827_l61_S
hand : LH
motionName : S1
beginFrame : 6827
slen : 61
slabel : S

frames : tableau de tableau de point 3D
"""
class PositionSequence:
    
    def __init__(self,filename):
        self.key = filename.split('\\')[len(filename.split('\\'))-1].split('.')[0] #Changer les \\ pour / sur Unix
        splited = self.key.split('_')
        self.hand = splited[0]
        self.motionName = splited[1]
        self.beginFrame = int(splited[2][1:])
        self.slen = int(splited[3][1:])
        self.slabel = splited[4]
        self.frames = []
        file = open(filename,'r')
        file.readline()
        for f in range(self.slen):
            frameF = file.readline().strip().split()
            frameI = []
            for p in range(0,len(frameF),3):
                point = [float(frameF[p]),float(frameF[p+1]),float(frameF[p+2])]
                frameI.append(point)
            self.frames.append(frameI)
        file.close() 
 

In [3]:
def files_to_sequences(folder='', traces=False) :
    """
    Transforme le contenu des fichiers du repertoire recursivement en tableau de PositionSequence. 
    Affiche des traces sur l'avancement si demande.
    """
    sequences = []
    files = glob.glob(folder + '/*.csv') #Changer les \ pour / sur Unix
    total = len(files)

    i = 1
    for file in files:
        try:
            sequences.append(PositionSequence(file))
        except: pass
        if traces:
            print('Chargement des donnees :', i, 'sur', total, end='\r')
            i += 1
    return sequences

def print_time_taken(start, msg):
    """
    Affiche le temps pris par une ou plusieurs commandes.
    @param start temps de debut donne par time.clock()
    @param msg message a afficher
    """
    print('%-50s : %.3f secondes' % (msg, time.time() - start))

In [4]:
start = time.time()
seq = files_to_sequences('PositionHandJoints', traces=True)
print_time_taken(start, 'time of loading') ; 

time of loading                                    : 7.488 secondes


> __Question 2 (manipulation d'une instance de classe)__ : En utilisant la structure de la classe PositionSequence, afficher les données de la première séquence de mouvement : main concernée, numéro du mouvement dont elle est issue, frame de démarrage, longueur, classification et position 3D des articulation à la premiere frame.

Affichage des données de la sequence 0
Main : LH
Sequence de mouvement : S10
Numero de la frame de debut : 1542
Longueur de la sequence : 93
Etiquette (classification) : Pi
Position 3D des articulations a la premiere frame :  [[27.0, 1.99808e-06, -4.59389e-07], [29.6252, -2.57866, 0.742169], [33.2855, -5.08981, 3.42089], [35.4144, -6.67086, 6.08897], [36.4832, -7.48297, 7.68688], [38.2187, -8.13014, 9.49917], [29.8343, -1.71399, 1.00364], [34.3001, -3.75315, 3.53836], [36.7106, -5.59238, 6.85252], [37.8287, -6.37284, 8.97365], [39.5935, -7.70144, 11.115], [30.0434, -0.849315, 1.26513], [34.8674, -1.81003, 4.28938], [37.4909, -3.64925, 7.89051], [38.5459, -4.9856, 10.1386], [39.5524, -6.44783, 12.2883], [30.1168, 0.228646, 1.44883], [34.9643, -0.0690782, 4.6581], [37.2336, -1.64061, 8.17652], [38.1882, -2.92555, 10.1032], [39.5791, -4.26917, 12.0785], [28.28, 1.49801, 1.55059], [29.7608, 1.9007, 1.8688], [31.6076, 3.96778, 3.74822], [33.5502, 4.71002, 5.61753], [35.8019, 6.34544, 7.4041

## Apprentissage automatique pour les données de configuration des mains

Nous allons travailler sur les données de position des articulations des mains pour traiter un problème de __classification supervisée__ (attribution d'une "étiquette", d'un "label" à ces données en utilisant une classification réalisées manuellement au préalable). 

__Nous souhaitons savoir si l'on peut désormais se passer d'une classification faite à la main et se reposer entièrement sur un algorithme qui ferait le travail automatiquement.__

Nous cherchons donc à entraîner des outils de classification ("classifieurs") de façon à reconnaître la configuration manuelle réalisée pendant une frame. Pour cela, vous allez utiliser des algorithmes d'apprentissage automatiques déja implémentés dans la bibliothèque Python Sklearn sur les données étiquetées. 

Et cela après avoir effectué les traitements suivants : 

1. Extraction d'indicateurs pertinents de ces données
2. application des classifieurs sur ces indicateurs après avoir préalablement séparé les données en deux ensembles : __l'ensemble d'entrainement__ qui va permettre au classifieur d'améliorer ses prédictions (en utilisant les étiquettes trouvées manuellement) et __l'ensemble de test__ qui va nous permettre d'évaluer les résultats des classifications.


Mais avant cela, il nous faut avoir plus d'informations sur les données. 

> __Question 3 (Statistiques sur les données)__ : 
Réprésenter graphiquement :
> - Le nombre de séquences (fichiers csv) par main (gauche/droite)
> - Le nombre de frames par main
> - Le nombre de séquences par configuration
> - Le nombre de frames par configuration
> - Le nombre de frames par configuration pour chaque main
> - Le nombre de frames par fichier de mouvement (motionName)
> - d'autres idées ? N'hésitez pas ! 


Cela permettra de construire des ensembles d’apprentissage et de test équilibrés.

### Détermination des caractéristiques (features) de classification

Les positions absolues des articulations dans l’espace ne permettent pas, telles quelles, de classifier les configurations de la main. En effet, deux configurations identiques peuvent être réalisées à deux endroits différents de l’espace et ne seront donc pas considérées comme la même configuration en utilisant les positions absolues. Il faut donc réfléchir à un ensemble de caractéristiques calculables à partir de ces positions qui sera adaptées à la tâche de classification. Les distances entre les articulations et/ou les angles formés par ces articulations peuvent être utilisés. Il peut être intéressant de réfléchir à des (sous-)ensembles pertinents de ces features.

> __Question 4 (Distance)__ :
> - Ecrire une fonction `computeDist(point1, point2)` qui calcule la distance entre deux points définis par 3 coordonées dans l'espace `[x,y,z]`.
> - La fonction `computeAngle(point1, point2, center, eps =1e-03 )` est donnée. Cette fonction calcule l'angle en degré formé par deux points 3D relativement à un troisième point `center`. Si l'angle est inférieur à `eps`, il est considéré comme égal à 0.

Pour calculer les angles nous avons utilisé la formule suivante :

Appelons $A$ le point1, $B$ le point2 et $C$ le centre de l'angle $\theta$ : 

$ CA.CB = |CA||CB|cos(\theta) $

In [9]:
def computeDist(point1, point2):
    """
    Calcule la distance entre deux points définis par 3 coordonées dans l'espace
    @param point1 Premier point
    @param point2 Deuxième point
    """
    
    
    return 0.0
    
def computeAngle(point1, point2, center, eps =1e-03 ):
    """
    Calcule l'angle en degré formé par deux points au niveau d'un troisième point
    @param point1 Premier point
    @param point2 Deuxième point
    @param center Vertex de l'angle
    """
    num = (center[0] - point1[0])*(point2[0] - center[0]) + \
    (center[1] - point1[1])*(point2[1] - center[1]) + \
    (center[2] - point1[2])*(point2[2] - center[2])

    denom = computeDist(point1,center) * computeDist(center,point2)
    if denom == 0:
        return 0

    angleDegre = math.acos(abs(num)/denom) * 180 /math.pi
    if angleDegre < eps:
        angleDegre = 0
    if num > 0:
        angleDegre = 180 - angleDegre
    return angleDegre

> __Question 5 (Caractéristiques de classification)__ :
Déterminer 2-3 ensembles de caractéristiques ("feature set") pertinents en vous aidant de la figure montrant le squelette de la main pour avoir la position des articulations. Quels distances ou angles de la main permettraient de distinguer les configurations de la main ? 



## Préparation des données pour la classification 

Les questions suivantes (questions 6 à 10) sont à réaliser pour chaque ensemble de caractéristiques.


La matrice X contient les valeurs des features choisis pour l’ensemble des frames considérées pour la classification. X est de taille : nbFrames * nbFeatures.

Le vecteur y, quant à lui, contient les étiquettes (labels) de chaque frame et est donc de taille nbFrames. Dans le cas où les features choisis seraient de nature différentes (angles et distances par exemple), il peut être nécessaire de normaliser les
valeurs de la matrices X.


> __Question 6 (Construction des matrices X et y)__ :
> - Ecrire une fonction `normaliseX(Xmat)` qui prend une matrice (tableau de tableau) et qui normalise les données (soustraie la moyenne à chaque donnée et divise le tout par l'ecart type)
>  - Construire les matrices X et y pour chaque features set choisi

In [13]:
def normaliseX(Xmat):
    """
    Normalise les donnees (soustraie la moyenne et divise par l'ecart type)
    """
    
    return "todo"

Pour créer l'ensemble d'apprentissage qui va servir à entrainer nos algorithmes de classification, nous allons prendre 80% des données. Nous évaluerons nos classifications avec les 20% restants (ensemble de test). Il n’est évidemment pas raisonnable d’entraîner un modèle sur 100% des données et de faire les tests sur ces mêmes données. Les individus pris pour le test seront déjà connus par le classifieur et le test ne sera pas valable.

> __Question 7 (Construction des ensembles d’apprentissage et de test)__ : Construire pour chaque ensemble de features, X_train (données d'entrainement), X_test (données de test), y_train (etiquettes pour l'entrainement), y_test (étiquette pour le test). Pour cela, n'h'ésitez pas à vous renseigner sur la fonction `train_test_split` de sklearn.
> 
> Etant donné leur utilisation, quelles difficultés voyez-vous à la construction d'ensembles d'entrainement et de test pertinents ? 

Réponse : il faudrait évidemment être plus rigoureux sur les ensembles : il n'est pas pertinent de les extraire à un niveau frame (il faudrait plutôt prendre des séquences entières. Il faudrait aussi avoir une répartition correcte pour chaque main et une répartition proportionnelles des HC ce qui n'est pas le cas ici (si si l'on utilise train_test_separate, je crois que stratify marche aussi...)


In [1]:
from sklearn.model_selection import train_test_split


## Classification


La classification elle-même peut maintenant commencer ! Pour cela, vous allez utiliser des algorithmes prédéfinis dans la bibliothèque sklearn de Python et comparer leurs résultats. Vous pourrez vous renseigner de votre côté pour savoir comment ils fonctionnent. Ils seront considérés comme des boites noires dans ce TP. 

Quel que soit l'algorithme utilisé, la classification comporte deux étapes : __l'entrainement__ ("fit") pendant lequel l'algorithme règle ses paramètres pour parvenir à reconnaitre les configurations de l'ensemble d'entrainement et __la prédiction__ ("predict") pendant laquelle l'algorithme tente de prédire les configurations manuelles sur un ensemble inconnu grâce au paramétrage réalisé pendant l'entrainement. Lors de cette deuxième phase, nous pourrons comparer les résultats obtenus avec les résultats attendus et évaluer la qualité de notre classifieur.


> __Question 8 (Classification)__ : Utiliser des algorithmes de classification (e.g. régression logistique, Naive Bayes, KNN, SVM) entraînés sur le training set pour reconnaître les configurations réalisées sur le test set.


In [141]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

## Analyse des résultats

Certaines métriques permettent d'évaluer les résultats dans le cas d'une tache de classification automatique. Les métriques les plus utilisées sont l'"accuracy", la "precision" et le "recall". 

Le résultat d'une prédiction peut être classé en 4 catégories :
- Vrai positif (TP) pour une certaine classe. Par exemple : telle configuration a été correctement assignée à la classe "5".
- Vrai négatif (TN) pour une certaine classe. Par exemple : telle configuration n'a pas été étiquetée "4" et à raison !
- Faux positif (FP) pour une certaine classe. Par exemple : telle configuration a été assignée à la classe "5" alors qu'elle appartient à la classe "3".
- Faux négatif (FN) pour une certaine classe. Par exemple : telle configuration n'a pas été étiquetée "4" mais à tord !

Ici, on donne les formules de chaque métrique pour une classe A. Comme notre problème est un problème multiclasse, il faudra faire une moyenne (par exemple) des résultats pour une métriques sur toutes les classes pour avoir le résultat global. 

L'accuracy mesure la proportion de cas correctement classés par rapport au nombre total d'objets à classifier.

$accuracy_{A}= \frac{TP_A + TN_A}{TP_A+TN_A+FP_A+FN_A}$

La précision mesure la capacité du modèle à identifier correctement les instances d'une classe particulière.

$precision_{A} = \frac{TP_A}{TP_A+FP_A}$

Le recall mesure la capacité du modèle à identifier toutes les instances d'une classe particulière.

$recall_{A} = \frac{TP_A}{TP_A+FN_A}$

> __Question 9 (Résultats numériques)__ : Calculez les valeurs d’accuracy, de precision, de recall en comparant les configurations prédites
avec les configurations annotées (y_test). N'hésitez pas à utiliser les fonctions prévues dans `sklearn.metrics`.

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


> __Question 10 (Résultats graphiques)__ : Les matrices de confusion sont souvent utilisées pour savoir si des classes sont souvent confondues. Utilisez [ConfusionMatrixDisplay](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)
 pour tracer la matrice de confusion des différents classifieurs. 


In [3]:
from sklearn.metrics import ConfusionMatrixDisplay

